In [ ]:
!pip install -q cassio datasets langchain openai tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 742.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.9/221.9 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 44.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# importing packages
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings

#support fro datset retrieval with Huggingg face
from datasets import load_dataset

# with CassIO, the engine powering the Astra DB integration in LangChain you will also initialize the DB connection
import cassio

In [ ]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.2 MB/s eta 0:00:00


In [ ]:
from PyPDF2 import PdfReader

In [ ]:
# setup
ASTRA_DB_APPLICATION_TOKEN = "your token"
ASTRA_DB_ID = "you ID"

OPENAI_API_KEY = "api key"

In [ ]:
pdfreader = PdfReader('/content/artificial_intelligence_tutorial.pdf')

In [ ]:
from typing_extensions import Concatenate
# read text from odf
raw_text = ''
for i, page in enumerate(pdfreader.pages):
  content = page.extract_text()
  if content:
    raw_text += content

raw_text

'Artificial Intelligence   \n  i About the T utorial  \nThis tutorial provides introductory  knowledge on Artificial Intelligence. It would come to a \ngreat help if you are about to select Artificial Intelligence as a course subject. You can briefly \nknow about the areas of AI in which research is prospering.  \nAudience  \nThis tutorial is prepared for the students at beginner level who aspire to learn Artificial \nIntelligence.  \nPrerequisites  \nThe basic knowledge of Computer Science is mandatory. The knowledge of Mathematics, \nLanguages,  Science , Mechanical or Electrical engineering is a plus.  \nDisclaimer & Copyright  \n\uf0e3 Copyright 201 5 by Tutorials Point (I) Pvt. Ltd.  \nAll the content and graphics published in this e -book are the property of Tutorials Point (I) \nPvt. Ltd.  The user of this e -book is prohibited to reuse, retain, copy, distribute or republish \nany contents or a part of contents of this e -book in any manner without wri tten consent of \nthe publ

In [ ]:
# initialize connection to the database
cassio.init(token=ASTRA_DB_APPLICATION_TOKEN, database_id= ASTRA_DB_ID)

ERROR:cassandra.connection:Closing connection <AsyncoreConnection(135323092065760) 24218851-d4ea-4297-ade6-e06e2b638cb0-us-east1.db.astra.datastax.com:29042:783fb26a-3338-451b-b75b-783a6e89aea9> due to protocol error: Error from server: code=000a [Protocol error] message="Beta version of the protocol used (5/v5-beta), but USE_BETA flag is unset"


In [ ]:
# create langchaiin embeddings and llm objects
llm = OpenAI(openai_api_key = OPENAI_API_KEY)
embedding = OpenAIEmbeddings(openai_api_key= OPENAI_API_KEY)

In [ ]:
# langchain vector store creation(Backend by Astra DB )
astra_vector_store = Cassandra(
    embedding = embedding,
    table_name = "qa_mini_demo",
    session = None,
    keyspace = None,
)

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

# we need to split the text using character text splitter such that it should not increase token size
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 800,
    chunk_overlap = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [ ]:
texts[:50]

['Artificial Intelligence   \n  i About the T utorial  \nThis tutorial provides introductory  knowledge on Artificial Intelligence. It would come to a \ngreat help if you are about to select Artificial Intelligence as a course subject. You can briefly \nknow about the areas of AI in which research is prospering.  \nAudience  \nThis tutorial is prepared for the students at beginner level who aspire to learn Artificial \nIntelligence.  \nPrerequisites  \nThe basic knowledge of Computer Science is mandatory. The knowledge of Mathematics, \nLanguages,  Science , Mechanical or Electrical engineering is a plus.  \nDisclaimer & Copyright  \n\uf0e3 Copyright 201 5 by Tutorials Point (I) Pvt. Ltd.  \nAll the content and graphics published in this e -book are the property of Tutorials Point (I)',
 'Disclaimer & Copyright  \n\uf0e3 Copyright 201 5 by Tutorials Point (I) Pvt. Ltd.  \nAll the content and graphics published in this e -book are the property of Tutorials Point (I) \nPvt. Ltd.  The use

In [ ]:
# load the dataset into the vector db/store
astra_vector_store.add_texts(texts[:50])
print("Inserted %i headlines." % len(texts[:50]))
astra_vector_index = VectorStoreIndexWrapper(vectorstore=astra_vector_store)

Inserted 42 headlines.


In [ ]:
# Running the QA Cycle
first_question = True
while True:
    if first_question:
        query_text = input("\nEnter your question (or type 'quit' to exit): ").strip()
    else:
        query_text = input("\nWhat's your next question (or type 'quit' to exit): ").strip()

    if query_text.lower() == "quit":
        break

    if query_text == "":
        continue

    first_question = False

    print("\nQUESTION: \"%s\"" % query_text)
    answer = astra_vector_index.query(query_text, llm=llm).strip()
    print("ANSWER: \"%s\"\n" % answer)

    print("FIRST DOCUMENTS BY RELEVANCE:")
    for doc, score in astra_vector_store.similarity_search_with_score(query_text, k=4):
        print("    [%0.4f] \"%s ...\"" % (score, doc.page_content[:84]))


Enter your question (or type 'quit' to exit): how would you differ AI from ml?

QUESTION: "how would you differ AI from ml?"


ANSWER: "Artificial Intelligence (AI) and Machine Learning (ML) are closely related, but they are not the same thing. AI is an umbrella term that covers a wide range of technologies, including ML. AI is the broader concept of machines being able to carry out tasks in a way that we would consider “smart”. On the other hand, ML is a specific type of AI that enables machines to learn from their environment and improve their performance without explicit programming. AI is focused on creating systems that can reason and make decisions, while ML is focused on creating systems that can learn and improve from data."

FIRST DOCUMENTS BY RELEVANCE:
    [0.9157] "Applications of AI  
AI has been dominant  in various fields such as : 
 Gaming 
AI ..."
    [0.9142] "Difference between Human and Machine Intelligence  ................................  ..."
    [0.9122] "answer the specific  questions it is meant 
to solve.  A computer program with  AI c ..."
    [0.9121] "Artificial intelligence is 

ANSWER: "Robotics is the branch of engineering and computer science that deals with the design, construction, operation, and use of robots, as well as computer systems for their control, sensory feedback, and information processing. It includes the study of how robots move and interact with their environment, and how they are programmed to think and act. Common components of robots include locomotion, perception, problem solving, and artificial intelligence. Robot applications include a wide range of activities from manufacturing and assembly to exploration and remote surgery."

FIRST DOCUMENTS BY RELEVANCE:
    [0.9179] "What is Robotics?  ................................ ................................ ..."
    [0.9097] "Components of a Robot  ................................ ............................ ..."
    [0.9087] "with locomotion, perception , and problem solving . Artificial Intelligence   
  10  ..."
    [0.9029] " natural languag e understanding and translation  
 Visi

ANSWER: "You're welcome!"

FIRST DOCUMENTS BY RELEVANCE:
    [0.8628] "Ltd. provides no guarantee regarding the accuracy, timeliness or completeness of our ..."
    [0.8598] "Audience  ................................ ................................ ........ ..."
    [0.8597] "Disclaimer & Copyright  
 Copyright 201 5 by Tutorials Point (I) Pvt. Ltd.  
All th ..."
    [0.8574] " natural languag e understanding and translation  
 Vision, Virtual Reality   
 G ..."

What's your next question (or type 'quit' to exit): quit
